In this notebook galaxies of different types atre distinguished. 

In [2]:
#standard libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import matplotlib.cm as cm
import sys
import os
#wcs is incompabible with newest numpy thus below not used 
#from astropy import wcs
#to access astronomical images in fits format
from astropy.io import fits
#torch functions
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
#sklearn helper functions
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score,f1_score, log_loss
#xgboost for comparison
from xgboost import XGBClassifier
#logistic regression for comparison 
from sklearn.linear_model import LogisticRegression


Now loading the data on the classification and other properties of the objects to be classified. 

In [4]:
df=pd.read_csv('Stripe82_1_zoo1.csv',sep=',')
print(df.columns)

Index(['objid', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'run', 'rerun', 'camcol',
       'field', 'specobjid', 'class', 'subclass', 'redshift', 'plate', 'mjd',
       'fiberid', 'nvote', 'p_el', 'p_cw', 'p_acw', 'p_edge', 'p_dk', 'p_mg',
       'p_el_debiased', 'p_cs_debiased'],
      dtype='object')


In [5]:
print(df['class'].value_counts())

GALAXY    461
QSO         5
Name: class, dtype: int64


In [7]:
print(df['subclass'].value_counts())

STARFORMING              143
STARBURST                 21
AGN                       18
BROADLINE                  8
STARFORMING BROADLINE      2
STARBURST BROADLINE        1
AGN BROADLINE              1
Name: subclass, dtype: int64


In [6]:
print(df['nvote'].value_counts())

28    34
30    31
33    27
32    27
31    20
27    20
35    19
59    18
29    16
26    13
37    13
34    12
25    11
23    11
36    10
22     8
42     8
38     8
24     7
47     7
52     7
51     7
58     7
65     7
55     7
56     6
46     6
40     6
54     6
57     5
43     5
20     5
62     5
61     5
45     4
41     4
53     4
69     4
44     4
66     4
21     3
49     3
73     3
70     3
39     3
64     3
50     2
17     2
19     2
67     2
68     2
63     2
60     2
16     1
74     1
75     1
79     1
72     1
18     1
Name: nvote, dtype: int64


In [8]:
print(df['redshift'].describe())

count    466.000000
mean       0.105482
std        0.068600
min        0.020425
25%        0.049426
50%        0.102624
75%        0.140850
max        0.748432
Name: redshift, dtype: float64


In [16]:
print(df.iloc[:,20:29].columns)

Index(['p_el', 'p_cw', 'p_acw', 'p_edge', 'p_dk', 'p_mg', 'p_el_debiased',
       'p_cs_debiased'],
      dtype='object')


In [19]:
for i in range(20, 28):
    print(df.iloc[:,i].describe())

count    466.000000
mean       0.555440
std        0.279177
min        0.000000
25%        0.321000
50%        0.600000
75%        0.795250
max        1.000000
Name: p_el, dtype: float64
count    466.000000
mean       0.068227
std        0.164644
min        0.000000
25%        0.000000
50%        0.007000
75%        0.043000
max        1.000000
Name: p_cw, dtype: float64
count    466.000000
mean       0.076116
std        0.160698
min        0.000000
25%        0.000000
50%        0.020000
75%        0.069000
max        0.971000
Name: p_acw, dtype: float64
count    466.000000
mean       0.196697
std        0.206639
min        0.000000
25%        0.043500
50%        0.126000
75%        0.267000
max        1.000000
Name: p_edge, dtype: float64
count    466.000000
mean       0.077376
std        0.082146
min        0.000000
25%        0.019000
50%        0.058500
75%        0.108000
max        0.606000
Name: p_dk, dtype: float64
count    466.000000
mean       0.026034
std        0.082431
mi

Clearly not all well classified, thus need further cleaningb as input. Also need to read paper more. 

In [24]:
df2=df[(df.p_el_debiased>2/3) | (df.p_cs_debiased>2/3) ]
print(df2.shape)
print(df.shape)

(287, 28)
(466, 28)
